In [ ]:
import requests
import time
import json
import geojson

In [27]:
with open("route1.json", "r") as r:
    route = geojson.load(r)

In [28]:
msg = {"name": "AIOmessenger_ONE"}

In [ ]:
while True:
    for lon, lat in route["features"][0]["geometry"]["coordinates"]:
        msg["lat"] = lat
        msg["lon"] = lon
        requests.post("http://127.0.0.1:8002/producer/geostream", json=msg)
        time.sleep(0.1)

In [70]:
for lon, lat in route["features"][0]["geometry"]["coordinates"]:
    msg["lat"] = lat
    msg["lon"] = lon
    requests.post("http://127.0.0.1:8002/producer/geostream", json=msg)
    time.sleep(0.05)

In [49]:
response = requests.get("http://127.0.0.1:8003/ping")
response

<Response [200]>